In [184]:
%display latex

%runfile young_idempotent.py
%runfile diagonal_polynomial_ring.py
%runfile subspace.py
%runfile add_degree.py
%runfile diagram.py

SymmetricFunctions(QQ).inject_shorthands(verbose=False)

#%runfile compute_character.py

# Code

Calcul du Vandermonde associé au diagramme $\gamma$, où $\gamma$ est un partage (Partition) ou un diagramme (Diagram).

In [185]:
def vandermonde(gamma, r=0):
    n = gamma.size()
    if r == 0:
        r = 1
    P = DiagonalPolynomialRing(QQ, n, r, inert=1) 
    X = P.variables()
    Theta = P.inert_variables()
    return matrix([[x**i[1]*theta**i[0] for i in gamma.cells()] 
                   for x,theta in zip(X[0],Theta[0])]).determinant()

Fonctions qui nous renvoient des listes d'opérateurs par degrés pour toutes les variables (non inertes) de l'anneau de polynômes concerné.

In [186]:
def partial_derivatives(P):
    """
    Return the partial derivative functions in all the variables of `P`.
    
    INPUT:
    - `P` -- a diagonal polynomial ring
    
    """
    n = P.ncols()
    r = P.nrows()
    D = P.grading_set()
    X = P.derivative_variables()
    op = {}
    for i in range(r):
        op[D((-1 if j==i else 0 for j in range(r)))] = [attrcall("derivative", X[i,k]) for k in range(n)]
    return op

def polarization_operators(r, max_deg=1, side=None, row_symmetry=None):
    D = cartesian_product([ZZ for i in range(r)])
    return {D([-d if i==i1 else 1 if i==i2 else 0 for i in range(r)]):
            [attrcall("polarization", i1=i1, i2=i2, d=d, row_symmetry=row_symmetry)]
            for d in range(1, max_deg+1)
            for i1 in range(0, r)
            for i2 in range(0, r)
            if (i1<i2 if side == 'down' else i1!=i2)
           }

##############################################################

def steenrod_operators(r, degree=1, row_symmetry=None):
    D = cartesian_product([ZZ for i in range(r)])
    op = {}
    for i in range(r):
        op[D((-degree if j==i else 0 for j in range(r)))] = [
            attrcall("steenrod_op", i=i, k=degree+1, row_symmetry=row_symmetry)]
    return op

def diagonal_steenrod_operators(list_deg):
    r = len(list_deg[0])
    D = cartesian_product([ZZ for i in range(r)])
    op = {}
    for dx, dy in list_deg:
        op[D((-dx if j==0 else -dy if j==1 else 0 for j in range(r)))] = [
            attrcall("diagonal_steenrod", i1=0, i2=1, d1=dx, d2=dy)]
    return op

def higher_polarization_operators(r, max_deg=1, side=None, row_symmetry=None):
    D = cartesian_product([ZZ for i in range(r)])
    return {D([-d1 if i==i1 else d2 if i==i2 else 0 for i in range(r)]):
            [attrcall("higher_polarization", i1=i1, i2=i2, d1=d1, d2=d2, row_symmetry=row_symmetry)]
            for d1 in range(1, max_deg+1)
            for d2 in range(1, 3)
            for i1 in range(0, r)
            for i2 in range(0, r)
            if (i1<i2 if side == 'down' else i1!=i2)
           }

def multipolarization(list_deg, i2):
    r = len(list_deg[0])
    D = cartesian_product([ZZ for i in range(r)])
    return {D(-d[i]+1 if i==i2 else -d[i] for i in range(len(d))) : 
            [attrcall("multi_polarization", D=d, i2=i2)] for d in list_deg}

def symmetric_derivatives(list_deg, row_symmetry=None):
    r = len(list_deg[0])
    D = cartesian_product([ZZ for i in range(r)])
    return {D(-i for i in d) : [attrcall("symmetric_derivative", d=d, row_symmetry=row_symmetry)] 
            for d in list_deg}

In [187]:
def merge(dict1, dict2):
    result = dict1
    for key, value in dict2.iteritems():
        if key in result:
            result[key] += value
        else:
            result[key] = value
    return result

def antisymmetries(mu):
    mu = Partition(mu)
    return antisymmetries_of_tableau(mu.initial_tableau())

Projection sur les composantes isotypiques; prend en paramètre un sous-espace (Subspace) et un entier ou un partage. 

In [188]:
def IsotypicComponent(S, arg, use_antisymmetry=False):
    if isinstance(arg, Partition):
        list_partitions = [arg]
    elif isinstance(arg, Integer):
        list_partitions = Partitions(arg)
    else : 
        print("Error: arg should be a partition or an integer.")
    
    basis = S.basis()
    result = {}
    P1 = basis.values().pop()[0].parent()
    for nu in list_partitions:
        result_nu = {}
        if use_antisymmetry == True:
            antisymmetries = antisymmetries_of_tableau(nu.initial_tableau())
            P2 = DiagonalAntisymmetricPolynomialRing(P1._R, P1.ncols(), P1.nrows(), 
                                                 P1.ninert(), antisymmetries)
        for deg, value in basis.iteritems():
            if use_antisymmetry:
                gen = []
                for p in value:
                    temp = apply_young_idempotent(P2(p), nu)
                    if temp != 0: 
                        gen += [temp]
            else:
                gen = []
                for p in value:
                    temp = apply_young_idempotent(p, nu)
                    if temp != 0:
                        gen += [temp]
            if gen != []:
                result_nu[(deg, tuple(nu))] = Subspace(gen, {}).basis()[0]
        if result_nu != {}:
            result[nu] = Subspace(result_nu, operators={})
                
    if len(result.keys()) == 1:
        key = result.keys()[0]
        return result[key]
    else:
        return result

In [189]:
def add_degrees_isotypic(gen_deg, op_deg):
    D = cartesian_product([ZZ for i in range(len(gen_deg[0]))])
    return D(gen_deg[0])+D(op_deg), gen_deg[1]

def add_degrees_symmetric(gen_deg,op_deg):
    D = cartesian_product([ZZ for i in range(len(gen_deg[0]))])
    d = D(gen_deg[0])+D(op_deg)
    return D(sorted(d, reverse=True)), gen_deg[1]

def add_degrees_test(gen_deg, op_deg):
    return gen_deg+op_deg

Création de l'espace polarisé par les opérateurs donnés en paramètres.

In [190]:
def PolarizedSpace(S, operators, add_degrees=add_degrees_isotypic):
    if isinstance(S, dict):
        return {key : PolarizedSpace(value, operators, add_degrees=add_degrees)
                for key, value in S.iteritems()}
    else:
        basis = S.basis()
        basis_element = basis.values().pop()[0]
        P1 = basis_element.parent()
        r = len(op_pol.keys().pop())
        row_symmetry = op_pol.values().pop()[0].kwds['row_symmetry']
        if row_symmetry == "permutation":
            add_degrees = add_degrees_symmetric
        D = cartesian_product([ZZ for i in range(r)])
        generators = {}

        if isinstance(P1, DiagonalAntisymmetricPolynomialRing):
            P2 = DiagonalAntisymmetricPolynomialRing(P1._R, P1.ncols(), r , P1.ninert(), P1.antisymmetries())
            for key, value in basis.iteritems():
                d = (D((key[0][0] if i==0 else 0 for i in range(0,r))), key[1])
                generators[d] = tuple(reduce_antisymmetric_normal(P2(b), 
                                                      b.parent().ncols(), 
                                                      b.parent().nrows()+b.parent().ninert(), 
                                                      b.antisymmetries()) for b in value)
        else :
            P2 = DiagonalPolynomialRing(P1._R, P1.ncols(), r , P1.ninert())
            for key, value in basis.iteritems():
                d = (D((key[0][0] if i==0 else 0 for i in range(0,r))), key[1])
                generators[d] = tuple(P2(b) for b in value)
        return Subspace(generators, operators, add_degrees=add_degrees)

Calcul de l'image de S (Subspace) par les opérateurs donnés.

In [191]:
def Range(S, operators, add_degrees=add_degrees_isotypic):
    if isinstance(S, dict):
        return {key : Range(value, operators, add_degrees=add_degrees)
                for key, value in S.iteritems()}

    result = {}
    basis = S.basis()
    for key, b in basis.iteritems():
        result = merge(result, {add_degrees(key, deg): 
                                     [op(p) for p in b for op in op_list if op(p)!=0] 
                                     for deg, op_list in operators.iteritems()})    
    if result != {} :
        return Subspace(result, {}, add_degrees) #{} <-> operators
    else :
        return None

Calcul du caractère à partir d'un sous-espace (S) d'éléments projetés sur les composantes isotypiques.

In [192]:
def character(S, left_basis=s, right_basis=s, row_symmetry=None):
    if isinstance(S, dict):
        return sum(character(V,
                             left_basis=left_basis, right_basis=right_basis, row_symmetry=row_symmetry) 
                   for V in S.values())
    else:
        basis = S.basis()
        basis_element = basis.values().pop()[0]
        P = basis_element.parent()
        n = P.ncols()
        r = P.nrows()
        
        charac = 0
        if row_symmetry != "permutation":
            q = PolynomialRing(QQ,'q',r).gens()

        for nu in Partitions(n):
            basis_nu = {}
            charac_nu = 0
            # Get the nu_isotypic part of the basis
            for key, value in basis.iteritems():
                if Partition(key[1]) == nu:
                    basis_nu[key[0]] = value

            # Use monomials to compute the character
            if row_symmetry == "permutation":
                for deg, b in basis_nu.iteritems():
                    charac_nu += sum(m(Partition(deg)) for p in b)
                if charac_nu != 0 :
                    if left_basis == s :
                        charac_nu = s(charac_nu).restrict_partition_lengths(r,exact=False)
                    elif left_basis != m :
                        charac_nu = left_basis(charac_nu)

            # Or use directly the degrees
            else:
                for deg, b in basis_nu.iteritems():
                    charac_nu += sum(prod(q[i]**deg[i] for i in range(0,len(deg))) for p in b)
                if charac_nu != 0 :
                    if left_basis == s :
                        charac_nu = s.from_polynomial(charac_nu).restrict_partition_lengths(r,exact=False)           
                    else:
                        charac_nu = left_basis.from_polynomial(charac_nu)

            # Make the tensor product with s[nu]
            if charac_nu != 0:
                charac += tensor([charac_nu, right_basis(s(nu))])
        return charac

In [193]:
def character_quotient(M, N, n, r, left_basis=s, right_basis=s): # a corriger
    b_tot = M.basis()
    b_ideal = N.basis()
    charac = 0
    q = PolynomialRing(QQ,'q',r).gens()
    
    for nu in Partitions(n):
        basis_nu_tot = {}
        basis_nu_ideal = {}
        charac_nu = 0
        # Get the nu_isotypic part of the bases
        for key, value in b_tot.iteritems():
            if Partition(key[1]) == nu:
                basis_nu_tot[key[0]] = value
        for key, value in b_ideal.iteritems():
            if Partition(key[1]) == nu:
                basis_nu_ideal[key[0]] = value
                
        # Use the degrees to compute the character
        for deg, b in basis_nu_tot.iteritems():
            charac_nu += sum(prod(q[i]**deg[i] for i in range(0,len(deg))) for p in b)
        for deg, b in basis_nu_ideal.iteritems():
            charac_nu -= sum(prod(q[i]**deg[i] for i in range(0,len(deg))) for p in b)
        if charac_nu != 0 :
            if left_basis == s :
                charac_nu = s.from_polynomial(charac_nu).restrict_partition_lengths(r,exact=False)           
            else:
                charac_nu = left_basis.from_polynomial(charac_nu)      
            # Make the tensor product with s[nu]
            charac += tensor([charac_nu, right_basis(s(nu))])
            
    return charac

Factorisation du produit tensoriel par représentation de $S_n$. Et calcul de la dimension.

In [194]:
def factorise(f, n):
    SymmetricFunctions(QQ).s()
    supp = sorted(f.support())
    result = {}
    res = []
    for mu in Partitions(n):
        result[mu] = []
        for (a, b), c in zip(supp, f.coefficients()):
            if b == mu :
                result[mu] += [(a,c)]
    result2 = [(mu,sum(c*s(nu) for (nu,c) in result[mu])) for mu in result.keys()]
    for a, b in result2:
        if b!=0:
            print a
            show(b)
        
def dimension(f, n):
    supp = sorted(f.support())
    result = {}
    res = []
    for mu in Partitions(n):
        result[mu] = []
        for (a, b), c in zip(supp, f.coefficients()):
            if b == mu :
                result[mu] += [(a,c)]
    result2 = [(mu,sum(c*s(nu) for (nu,c) in result[mu]).expand(1, alphabet=['q'])) for mu in result.keys()]
    q = result2[0][1].parent().gens()[0]
    return [(tuple(a), b.subs({q:1})) for a,b in result2]

def dimension2(f, n):
    q,t = QQ['q,t'].gens()
    supp = sorted(f.support())
    result = {}
    res = []
    for mu in Partitions(n):
        result[mu] = []
        for (a, b), c in zip(supp, f.coefficients()):
            if b == mu :
                result[mu] += [(a,c)]
    result2 = [(mu,sum(c*s(nu) for (nu,c) in result[mu]).expand(2, alphabet=[q,t])) for mu in result.keys()]
    return [(tuple(a), b.subs({q:1,t:1})) for a,b in result2]

In [ ]:
def compute_character(mu, use_antisymmetry=True, row_symmetry="permutation"):
    n = Integer(mu.size())
    # Determinant computation
    v = vandermonde(mu)
    # Span by derivatives
    generator = {v.multidegree() : [v]}
    list_op = partial_derivatives(v.parent())
    V = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
    # Projection on isotypic components
    V_iso = IsotypicComponent(V, n, use_antisymmetry=use_antisymmetry)
    # Polarization
    r = n-1
    deg = v.degree()
    if deg == 0:
        deg = 1
    op_pol = polarization_operators(r, deg, row_symmetry=row_symmetry)
    V_pol = PolarizedSpace(V_iso, op_pol)
    
    # character
    return character(V_pol, row_symmetry=row_symmetry)

In [195]:
#import pdb

In [196]:
#pdb.pm()

# Exemples de calculs

### Cas du partage 21 - sans antisymétries

Calcul du déterminant de Vandermonde $\mathcal{V}_\lambda$ associé au partage $\lambda = 21$.

In [197]:
n = 3
v = vandermonde(Partition([2,1]))
v

-x01*theta00 + x02*theta00 + x00*theta01 - x02*theta01 - x00*theta02 + x01*theta02

Espace généré par $\mathcal{V}_\lambda$ et ses dérivées partielles.

In [198]:
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
V = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
V.basis()

{(0,): (theta01 - theta02, theta00 - theta02),
 (1,): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

Projection sur les composantes isotypiques.

In [199]:
V_iso = IsotypicComponent(V, n)
for value in V_iso.values():
    show(value.basis())

{((1,),
  (1,
   1,
   1)): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

{((0,), (2, 1)): (-theta00 + theta02,)}

On ajoute un jeu de variables supplémentaire et on polsarise.

In [200]:
r = 2 
deg = 2
op_pol = polarization_operators(r, deg)
V_pol = PolarizedSpace(V_iso, op_pol)
for value in V_pol.values():
    show(value.basis())

{((0, 1),
  (1,
   1,
   1)): (-x11*theta00 + x12*theta00 + x10*theta01 - x12*theta01 - x10*theta02 + x11*theta02,),
 ((1, 0),
  (1,
   1,
   1)): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

{((0, 0), (2, 1)): (theta00 - theta02,)}

Calcul du bicaractère.

In [201]:
character(V_pol)

s[] # s[2, 1] + s[1] # s[1, 1, 1]

### Cas du partage 21 - avec antisymétries

In [202]:
v = vandermonde(Partition([2,1]))
v

-x01*theta00 + x02*theta00 + x00*theta01 - x02*theta01 - x00*theta02 + x01*theta02

Espace généré par $\mathcal{V}_\lambda$ et ses dérivées partielles.

In [203]:
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
V = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
V.basis()

{(0,): (theta01 - theta02, theta00 - theta02),
 (1,): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

Projection sur les composantes isotypiques.

In [204]:
V_iso = IsotypicComponent(V, 3, use_antisymmetry=True)
for value in V_iso.values():
    show(value.basis())

{((1,), (1, 1, 1)): (x00*theta01,)}

{((0,), (2, 1)): (theta00,)}

On ajoute un jeu de variables supplémentaire et on polsarise.

In [205]:
r = 2 
deg = 2
op_pol = polarization_operators(r, deg)
V_pol = PolarizedSpace(V_iso, op_pol)
for value in V_pol.values():
    show(value.basis())

{((0, 1), (1, 1, 1)): (x10*theta01,), ((1, 0), (1, 1, 1)): (x00*theta01,)}

{((0, 0), (2, 1)): (theta00,)}

Calcul du bicaractère

In [206]:
character(V_pol)

s[] # s[2, 1] + s[1] # s[1, 1, 1]

### Tout en un

In [207]:
mu = Partition([4,1])
n = mu.size()
v = vandermonde(mu)
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
V = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
V_iso = IsotypicComponent(V, n, use_antisymmetry=True)

r = n-1
deg = degree_vandermonde(mu)
if deg == 0:
    deg = 1
op_pol = polarization_operators(r, deg, row_symmetry="permutation")
V_pol = PolarizedSpace(V_iso, op_pol)

character(V_pol, row_symmetry="permutation")

s[] # s[4, 1] + s[1] # s[3, 1, 1] + s[1] # s[3, 2] + s[1, 1] # s[2, 1, 1, 1] + s[1, 1] # s[2, 2, 1] + s[1, 1, 1] # s[1, 1, 1, 1, 1] + s[2] # s[2, 2, 1] + s[2] # s[3, 1, 1] + s[2] # s[3, 2] + s[2, 1] # s[2, 1, 1, 1] + s[2, 1] # s[2, 2, 1] + s[3] # s[2, 1, 1, 1] + s[3] # s[2, 2, 1] + s[3] # s[3, 1, 1] + s[3, 1] # s[1, 1, 1, 1, 1] + s[3, 1] # s[2, 1, 1, 1] + s[4] # s[2, 1, 1, 1] + s[4] # s[2, 2, 1] + s[4, 1] # s[1, 1, 1, 1, 1] + s[5] # s[2, 1, 1, 1] + s[6] # s[1, 1, 1, 1, 1]

In [208]:
def compute_caracter(mu, use_antisymmetry=True, row_symmetry="permutation"):
    n = Integer(mu.size())
    # Determinant computation
    v = vandermonde(mu)
    # Span by derivatives
    generator = {v.multidegree() : [v]}
    list_op = partial_derivatives(v.parent())
    V = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
    # Projection on isotypic components
    V_iso = IsotypicComponent(V, n, use_antisymmetry=use_antisymmetry)
    # Polarization
    r = n-1
    deg = degree_vandermonde(mu)
    if deg == 0:
        deg = 1
    op_pol = polarization_operators(r, deg, row_symmetry=row_symmetry)
    V_pol = PolarizedSpace(V_iso, op_pol)
    
    # character
    return character(V_pol, row_symmetry=row_symmetry)

In [209]:
compute_character(Partition([4,1]))

s[] # s[4, 1] + s[1] # s[3, 1, 1] + s[1] # s[3, 2] + s[1, 1] # s[2, 1, 1, 1] + s[1, 1] # s[2, 2, 1] + s[1, 1, 1] # s[1, 1, 1, 1, 1] + s[2] # s[2, 2, 1] + s[2] # s[3, 1, 1] + s[2] # s[3, 2] + s[2, 1] # s[2, 1, 1, 1] + s[2, 1] # s[2, 2, 1] + s[3] # s[2, 1, 1, 1] + s[3] # s[2, 2, 1] + s[3] # s[3, 1, 1] + s[3, 1] # s[1, 1, 1, 1, 1] + s[3, 1] # s[2, 1, 1, 1] + s[4] # s[2, 1, 1, 1] + s[4] # s[2, 2, 1] + s[4, 1] # s[1, 1, 1, 1, 1] + s[5] # s[2, 1, 1, 1] + s[6] # s[1, 1, 1, 1, 1]

### Cas du diagramme 013

In [210]:
v = vandermonde(Diagram([(0,0),(1,0),(3,0)]))
v.factor()

(x01 - x02) * (-x00 + x01) * (x00 - x02) * (x00 + x01 + x02)

In [211]:
v.parent()

Diagonal Polynomial Ring with 1 rows of 3 variables and inert variables over Rational Field

On crée le sous-espace engendré par $\mathcal{V}_\gamma$ et ses dérivées partielles. On peut ajouter les opérateurs de Steenrod sans changement du résultat.

In [212]:
generator = {v.multidegree() : [v]}
list_op = merge(merge(partial_derivatives(v.parent()), steenrod_operators(1, 1)), steenrod_operators(1, 2))

In [213]:
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
W1.basis()

{(0,): (1,),
 (1,): (x00, x01, x02),
 (2,): (x00^2 - x02^2, x01^2 - x02^2, -x00*x01 + x00*x02, -x00*x01 + x01*x02),
 (3,): (1/3*x00^3 - 1/3*x01^3 - x00*x02^2 + x01*x02^2,
  -1/3*x00^3 + x00*x01^2 - x01^2*x02 + 1/3*x02^3,
  -x00^2*x01 + 1/3*x01^3 + x00^2*x02 - 1/3*x02^3),
 (4,): (-x00^3*x01 + x00*x01^3 + x00^3*x02 - x01^3*x02 - x00*x02^3 + x01*x02^3,)}

In [214]:
character(IsotypicComponent(W1, 3))

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + 2*s[2] # s[2, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

Appliquons $\sum_i \partial_{x_i}$ et $\sum_i x_i \partial_{x_i}^2$ à $\mathcal{V}_{\gamma}$. 

In [215]:
x00, x01, x02 = P1.variables()[0]
v.symmetric_derivative((1,))

-3*x00^2*x01 + 3*x00*x01^2 + 3*x00^2*x02 - 3*x01^2*x02 - 3*x00*x02^2 + 3*x01*x02^2

In [216]:
v.steenrod_op(0, 2)

-6*x00^2*x01 + 6*x00*x01^2 + 6*x00^2*x02 - 6*x01^2*x02 - 6*x00*x02^2 + 6*x01*x02^2

Polarisation

In [217]:
op_pol = polarization_operators(2, max_deg=v.degree())
W2 = PolarizedSpace(IsotypicComponent(W1, 3), op_pol)
character(W2)

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

In [218]:
list_degrees = [tuple(k1 if j==i1 else 0 for j in range(P2._r)) 
                for k1 in range(1, 3+1) for i1 in range(0, P2._r)]
#list_degrees += [(1,1),(1,2),(1,3),(2,1),(2,2),(2,3),(3,1),(3,2),(3,3)]
sym_diff = symmetric_derivatives(2, list_degrees)
#sym_diff = merge(merge(symmetric_derivatives(P2, list_degrees), 
#                       steenrod_operators(P2, 1)), steenrod_operators(P2, 2))
character(Range(W2, sym_diff))

s[] # s[3] + s[1] # s[2, 1] + s[1, 1] # s[1, 1, 1] + s[2] # s[2, 1] + s[3] # s[1, 1, 1]

In [219]:
charac = character(W2) - character(Range(W2, sym_diff))
charac

s[1] # s[3] + s[1, 1] # s[2, 1] + s[2] # s[2, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

Ou dans la base des monomiales

In [220]:
character(W2, left_basis=m) - character(Range(W2, sym_diff), left_basis=m)

m[1] # s[3] + 2*m[1, 1] # s[2, 1] + m[2] # s[2, 1] + m[2, 1] # s[1, 1, 1] + m[2, 1] # s[2, 1] + m[2, 2] # s[1, 1, 1] + m[3] # s[2, 1] + m[3, 1] # s[1, 1, 1] + m[4] # s[1, 1, 1]

On factorise le caractère par représentation de $S_3$.

In [221]:
factorise(charac, 3)

[1, 1, 1]


s[2, 1] + s[4]

[3]


s[1]

[2, 1]


s[1, 1] + s[2] + s[3]

### Cas du diagramme 024 : en cours de résolution

In [222]:
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
v.factor()

(x01 - x02) * (x01 + x02) * (-x00 + x01) * (x00 - x02) * (x00 + x02) * (x00 + x01)

### Tests pour 024

**Quotient puis polarisation** et test des dimensions (vérification du symétriseur de Young)

In [223]:
# Create M with V, partial derivatives and Steenrod in x
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
deg = v.degree()
generator = {v.multidegree() : [v]}
#list_op = partial_derivatives(1)
list_op = merge(steenrod_operators(1, 1), steenrod_operators(1, 2))
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
print "dimension V1:", W1.dimension()
W1 = IsotypicComponent(W1, Partition([1,1,1]))
#print dimension(character(W1, 3, 1), 3)
print

# Create ideal I*M in x using symmetric derivatives
list_degrees = [tuple((i,)) for i in range(1,4)]
#sym_diff = symmetric_derivatives(P1, list_degrees)
sym_diff = merge(steenrod_operators(1, 1), steenrod_operators(1, 2))
W2 = Range(W1, sym_diff)

# Print caracters in one set of variables
print "V1(X)"
show(character(W1))
print "V2(X)"
show(character(W2))
print "V1(X)-V2(X)"
show(character(W1)- character(W2))
show(character_quotient(W1, W2, 3, 1))

# Polarisation of M and I*M
r=2
op_pol = polarization_operators(2, max_deg=4)
op_pol2 = polarization_operators(2, max_deg=4, side="down")
list_degrees = [tuple((i,j)) for i in range(1,deg) for j in range(1,deg) if i+j!=0 and i+j<=deg]
list_degrees += [(0,1), (0,2), (1,0), (2,0)]
#op_pol2 = merge(op_pol2, diagonal_steenrod_operators(r, list_degrees))
op_pol = merge(merge(op_pol, steenrod_operators(r, 1)), steenrod_operators(r,2))
op_pol2 = merge(merge(op_pol2, multipolarization(r, list_degrees, 0)), multipolarization(r, list_degrees, 1))
#op_pol = merge(op_pol, multipolarization(r, list_degrees, 0))
#op_pol = merge(op_pol, multipolarization(r, list_degrees, 1))
W1 = PolarizedSpace(W1, op_pol)
W2 = PolarizedSpace(W2, op_pol2)

# Print caracters in two sets of variables
print "V1(X,Y)"
show(character(W1, left_basis=m))
print "V2(X,Y)"
show(character(W2, left_basis=m))
charac = character(W1, left_basis=m) - character(W2, left_basis=m)
print "V1(X,Y)-V2(X,Y)"
show(charac)
show(character_quotient(W1, W2, 3, 2))

dimension V1: 4

V1(X)


s[3] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V2(X)


s[3] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[5] # s[1, 1, 1]

V1(X)-V2(X)


...

V2(X,Y)


m[1, 1] # s[1, 1, 1] + 2*m[2, 1] # s[1, 1, 1] + 2*m[2, 2] # s[1, 1, 1] + m[3] # s[1, 1, 1] + 2*m[3, 1] # s[1, 1, 1] + m[3, 2] # s[1, 1, 1] + m[4] # s[1, 1, 1] + m[4, 1] # s[1, 1, 1] + m[5] # s[1, 1, 1]

V1(X,Y)-V2(X,Y)


m[2, 2] # s[1, 1, 1] + m[3, 1] # s[1, 1, 1] + m[3, 2] # s[1, 1, 1] + m[3, 3] # s[1, 1, 1] + m[4, 1] # s[1, 1, 1] + m[4, 2] # s[1, 1, 1] + m[5, 1] # s[1, 1, 1] + m[6] # s[1, 1, 1]

s[3, 1] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + s[6] # s[1, 1, 1]

**$V_1(X,Y)$ stable par dérivation partielle mais pas $V_2(X,Y)$**

In [230]:
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
deg = v.degree()
generator = {v.multidegree() : [v]}
list_op = merge(merge(partial_derivatives(v.parent()), steenrod_operators(1, 1)), steenrod_operators(1, 2))
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

list_degrees = [tuple(k1 if j==i1 else 0 for j in range(P1._r)) for k1 in range(1, 3+1) for i1 in range(0, P1._r)]
sym_diff = symmetric_derivatives(1, list_degrees)
sym_diff = merge(merge(sym_diff, steenrod_operators(1, 1)), steenrod_operators(1, 2))
W2 = Range(W1, sym_diff, add_degrees=add_degrees_test)

r=2
D = cartesian_product([ZZ for i in range(r)])
P2 = DiagonalPolynomialRing(QQ, 3, r)
op_pol = polarization_operators(r, max_deg=deg)
gen = {}
for key, value in W1.basis().iteritems():
    d = D((key[0] if i==0 else 0 for i in range(0,r)))
    gen[d] = [P2(b) for b in value]
W1 = Subspace(gen, op_pol, add_degrees=add_degree)
W12 = Subspace(W1.basis(), partial_derivatives(P2), add_degrees=add_degree)

gen = {}
for key, value in W2.basis().iteritems():
    d = D((key[0] if i==0 else 0 for i in range(0,r)))
    gen[d] = [P2(b) for b in value]
W2 = Subspace(gen, op_pol, add_degrees=add_degree)
W22 = Subspace(W2.basis(), partial_derivatives(P2), add_degrees=add_degree)

nu = Partition([2,1])
print 'V1'
show(character(IsotypicComponent(W1, nu), left_basis=s))
show(character(IsotypicComponent(W12, nu), left_basis=s))
print 
print 'V2'
show(character(IsotypicComponent(W2, nu), left_basis=s))
show(character(IsotypicComponent(W22, nu), left_basis=s))

V1


s[1] # s[2, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + 2*s[2, 1] # s[2, 1] + 3*s[3] # s[2, 1] + s[3, 1] # s[2, 1] + 3*s[4] # s[2, 1] + s[5] # s[2, 1]

s[1] # s[2, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + 2*s[2, 1] # s[2, 1] + 3*s[3] # s[2, 1] + s[3, 1] # s[2, 1] + 3*s[4] # s[2, 1] + s[5] # s[2, 1]


V2


s[1] # s[2, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + 2*s[2, 1] # s[2, 1] + 3*s[3] # s[2, 1] + 2*s[4] # s[2, 1]

s[1] # s[2, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + 2*s[2, 1] # s[2, 1] + 3*s[3] # s[2, 1] + 3*s[4] # s[2, 1]

**Polarisation puis quotient**

In [231]:
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
deg = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
list_op = merge(merge(list_op, steenrod_operators(1, 1)), steenrod_operators(1, 2))
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
W1 = IsotypicComponent(W1, Partition([1,1,1]))

print "V1(X)"
show(character(W1))

r=2
op_pol = polarization_operators(r, max_deg=deg, side="down")
op_pol = merge(merge(op_pol, steenrod_operators(r, 1)), steenrod_operators(r, 2))
W1 = PolarizedSpace(W1, op_pol)

print "V1(X,Y)"
show(character(W1))

list_degrees = [tuple((i,j)) for i in range(0, deg) for j in range(0, deg) if i+j>0 and i+j<deg]
sym_diff = symmetric_derivatives(r, list_degrees)
sym_diff = merge(merge(sym_diff, steenrod_operators(r, 1)), steenrod_operators(r, 2))
W2 = Range(W1, sym_diff, add_degrees=add_degrees_isotypic)

print "V2(X,Y)"
show(character(W2))

print "V1(X,Y)-V2(X,Y)"
charac = character(W1) - character(W2)
show(charac)

V1(X)


s[3] # s[1, 1, 1] + s[4] # s[1, 1, 1] + 2*s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V1(X,Y)


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + 2*s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + 2*s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V2(X,Y)


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + 2*s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + 2*s[5] # s[1, 1, 1]

V1(X,Y)-V2(X,Y)


s[6] # s[1, 1, 1]

**Tests** sur le modèle polarisation puis quotient

In [234]:
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
deg = v.degree()
generator = {v.multidegree() : [v]}
list_op1 = partial_derivatives(v.parent())
list_op2 = merge(steenrod_operators(1, 1), steenrod_operators(1, 2))
W1 = Subspace(generators=generator, operators=list_op2, add_degrees=add_degree)
W1 = IsotypicComponent(W1, Partition([1,1,1]))

print "V1(X)"
show(character(W1))

r=2
P2 = DiagonalPolynomialRing(QQ, 3, r)
op_pol = polarization_operators(r, max_deg=deg, side="down")
op_pol = merge(merge(op_pol, steenrod_operators(r, 1)), steenrod_operators(r, 2))
W1 = PolarizedSpace(W1, op_pol)

print "V1(X,Y)"
show(character(W1))

sym_diff = {}
list_degrees = [tuple((i,j)) for i in range(0, deg) for j in range(0, deg) if i+j>0 and i+j<deg]
sym_diff1 = symmetric_derivatives(r, list_degrees)
sym_diff2 = steenrod_operators(r, 1)
sym_diff3 = steenrod_operators(r, 2)
sym_diff4 = merge(symmetric_derivatives(r, list_degrees), steenrod_operators(r, 1))
W2 = Range(W1, sym_diff2, add_degrees=add_degrees_isotypic)
W3 = Range(W2, sym_diff2, add_degrees=add_degrees_isotypic)
W4 = Range(W1, sym_diff1, add_degrees=add_degrees_isotypic)
W5 = Range(W4, sym_diff1, add_degrees=add_degrees_isotypic)

#print "test"
#W2bis = Range(W1bis, steenrod_operators(1, 1))
#show(character(W2bis, 3, 1))

print "V2(X,Y)"
show(character(W2))
print "V3(X,Y) steenrod x2"
show(character(W3, left_basis=s))
print "V4(X,Y) deriv sym"
show(character(W4, left_basis=s))
print "V5(X,Y) deriv sym x2"
show(character(W5))

print "2x steen 2x deriv sym"
W34 = Subspace(merge(W3.basis(), W5.basis()), {})
show(character(W34, left_basis=m))
show(character(W1) - character(W34))

print "V1(X,Y)-V2(X,Y)"
#charac = character(W1) - character(W3) - character(W2bis)
show(charac)
charac = character(W1) - character(W3) - character(W4) + character(W5)
show(charac)

V1(X)


s[3] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V1(X,Y)


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + 2*s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V2(X,Y)


...

m[1, 1] # s[1, 1, 1] + 2*m[2, 1] # s[1, 1, 1] + 2*m[2, 2] # s[1, 1, 1] + m[3] # s[1, 1, 1] + 3*m[3, 1] # s[1, 1, 1] + m[4] # s[1, 1, 1]

s[2, 2] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V1(X,Y)-V2(X,Y)


s[2, 2] # s[1, 1, 1] - s[3, 1] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + s[6] # s[1, 1, 1]

s[2, 2] # s[1, 1, 1] - s[3, 1] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + s[6] # s[1, 1, 1]

In [235]:
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
deg = v.degree()
generator = {v.multidegree() : [v]}
list_op = {}
#list_op = partial_derivatives(1)
#list_op = merge(merge(list_op, steenrod_operators(1, 1)), steenrod_operators(1, 2))
list_op = merge(list_op, steenrod_operators(1, 1))
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
W1 = IsotypicComponent(W1, Partition([1,1,1]))

print "V1(X)"
show(character(W1))

r=2
op_pol = polarization_operators(r, max_deg=2)
#op_pol = merge(op_pol, steenrod_operators(r, 1)), steenrod_operators(r, 2))
W1 = PolarizedSpace(W1, op_pol)

print "V1(X,Y)"
show(character(W1))

list_degrees = [tuple((i,j)) for i in range(0, deg) for j in range(0, deg) if i+j>0 and i+j<deg]
#sym_diff = symmetric_derivatives(r, list_degrees)
#sym_diff = merge(merge(sym_diff, steenrod_operators(r, 1)), steenrod_operators(r, 2))
sym_diff = steenrod_operators(r, 1)
W2 = Range(W1, sym_diff, add_degrees=add_degrees_isotypic)
W3 = Range(W1, symmetric_derivatives(r, list_degrees), add_degrees=add_degrees_isotypic )

print "V2(X,Y)"
show(character(W2))
print "V3(X,Y)"
show(character(W3))

print "V1(X,Y)-V2(X,Y)"
charac = character(W1) - character(W2)
show(charac)

V1(X)


s[3] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V1(X,Y)


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + 2*s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V2(X,Y)


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] - s[2, 2] # s[1, 1, 1] + s[3] # s[1, 1, 1] + 2*s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + s[5] # s[1, 1, 1]

V3(X,Y)


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[5] # s[1, 1, 1]

V1(X,Y)-V2(X,Y)


s[2, 2] # s[1, 1, 1] + s[6] # s[1, 1, 1]

On vérifie ici aussi que **$V_1(X,Y)$ stable par dérivation partielle mais pas $V_2(X,Y)$**. 

(*Ok pour les partages $3$ et $21$ mais problème avec $111$.*)

Projection sur les composantes isotypiques après polarisation et ajout de tous les opérateurs **sans optimisation**. Et vérification des dimensions.  

In [245]:
r = 2
P1 = DiagonalPolynomialRing(QQ, 3, r)
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
deg = v.degree()
generator = {P1._grading_set((v.degree(),0)) : [P1(v)]}
list_op = partial_derivatives(P1)
list_op = merge(merge(list_op, steenrod_operators(r, 1)), steenrod_operators(r, 2))
list_op = merge(list_op, polarization_operators(r, max_deg=4))
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

list_degrees = [tuple((i,j)) for i in range(0,deg+1) for j in range(0,deg+1) if i+j!=0 and i+j<deg+1]
sym_diff = symmetric_derivatives(r, list_degrees)
#sym_diff = merge(merge(sym_diff, steenrod_operators(P1, 1)), steenrod_operators(P1, 2))
W2 = Range(W1, sym_diff, add_degrees=add_degrees_test)

print 'dimension', W1.dimension()
W1 = IsotypicComponent(W1, 3)
print "V1(X,Y)"
show(character(W1))
print 'dimension', dimension2(character(W1), 3)

W2 = IsotypicComponent(W2, 3)
print "V2(X,Y)"
show(character(W2))

charac = character(W1) - character(W2)
print "V1(X,Y)-V2(X,Y)"
charac

dimension 152
V1(X,Y)


s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + 2*s[2] # s[3] + s[2, 1] # s[1, 1, 1] + 2*s[2, 1] # s[2, 1] + s[3] # s[1, 1, 1] + 3*s[3] # s[2, 1] + s[3] # s[3] + 2*s[3, 1] # s[1, 1, 1] + s[3, 1] # s[2, 1] + s[4] # s[1, 1, 1] + 3*s[4] # s[2, 1] + s[4, 1] # s[1, 1, 1] + 2*s[5] # s[1, 1, 1] + s[5] # s[2, 1] + s[6] # s[1, 1, 1]

dimension [((1, 1, 1), 41), ((3,), 13), ((2, 1), 49)]
V2(X,Y)


s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + s[2] # s[3] + s[2, 1] # s[1, 1, 1] + s[2, 1] # s[2, 1] + s[3] # s[1, 1, 1] + 2*s[3] # s[2, 1] + s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[4] # s[2, 1] + s[5] # s[1, 1, 1]

V1(X,Y)-V2(X,Y)


s[2] # s[3] + s[2, 1] # s[2, 1] + s[3] # s[2, 1] + s[3] # s[3] + s[3, 1] # s[1, 1, 1] + s[3, 1] # s[2, 1] + 2*s[4] # s[2, 1] + s[4, 1] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[5] # s[2, 1] + s[6] # s[1, 1, 1]

**Test : ajout de nouveaux opérateurs** avec polarisation puis quotient

In [249]:
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
deg = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
#list_op = merge(merge(list_op, steenrod_operators(1, 1)), steenrod_operators(1, 2))
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
W1 = IsotypicComponent(W1, Partition([1,1,1]))

print "V1(X)"
show(character(W1))

r=2
op_pol = polarization_operators(r, deg)
#op_pol = merge(op_pol, higher_polarization_operators(r))
#op_pol = merge(op_pol, {D([-2,2]): [functools.partial(P2.higher_polarization, i1=0, i2=1, d1=2, d2=2)]})
W1 = PolarizedSpace(W1, op_pol)

print "V1(X,Y)"
show(character(W1))

#list_degrees = [tuple(k1 if j==i1 else 0 for j in range(r)) for k1 in range(1, 3+1) for i1 in range(0, P2._r)]
list_degrees = [tuple((i,j)) for i in range(0,deg+1) for j in range(0,deg+1) if i+j!=0 and i+j<deg+1]
sym_diff = symmetric_derivatives(r, list_degrees)
#sym_diff = merge(merge(sym_diff, steenrod_operators(r, 1)), steenrod_operators(r, 2))
W2 = Range(W1, sym_diff, add_degrees=add_degrees_isotypic)

print "V2(X,Y)"
show(character(W2))

charac = character(W1) - character(W2)
print "V1(X,Y)-V2(X,Y)"
charac

V1(X)


s[3] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V1(X,Y)


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + 2*s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + 2*s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

V2(X,Y)


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[5] # s[1, 1, 1]

V1(X,Y)-V2(X,Y)


s[3, 1] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

Construction du **quotient avec des générateurs différents** : utilisant les déplacements des cases. Avec les opérateurs actuels, on retrouve bien le caractère obtenu en quotientant avant de polariser. 

In [250]:
r = 2
D = cartesian_product([ZZ for i in range(r)])
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]), r=r)
deg = 6
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
list_op = merge(merge(list_op, steenrod_operators(r, 1)), steenrod_operators(r, 2))
list_op = merge(list_op, polarization_operators(r, max_deg=4))
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

v014 = vandermonde(Diagram([(0,0),(1,0),(4,0)]), r=r)
generator014 = {v014.multidegree() : [v014]}
W1_014 = Subspace(generators=generator014, operators=list_op, add_degrees=add_degree)
W1_014 = IsotypicComponent(W1_014, 3)
print "V1_014(X,Y)"
show(character(W1_014))

v013 = vandermonde(Diagram([(0,0),(1,0),(3,0)]), r=r)
generator013 = {v013.multidegree() : [v013]}
W1_013 = Subspace(generators=generator013, operators=list_op, add_degrees=add_degree)
W1_013 = IsotypicComponent(W1_013, 3)
print "V1_013(X,Y)"
show(character(W1_013))

v012 = vandermonde(Diagram([(0,0),(1,0),(2,0)]), r=r)
generator012 = {v012.multidegree() : [v012]}
W1_012 = Subspace(generators=generator012, operators=list_op, add_degrees=add_degree)
W1_012 = IsotypicComponent(W1_012, 3)
print "V1_012(X,Y)"
show(character(W1_012))

v023 = vandermonde(Diagram([(0,0),(2,0),(3,0)]), r=r)
generator023 = {v023.multidegree() : [v023]}
W1_023 = Subspace(generators=generator023, operators=list_op, add_degrees=add_degree)
W1_023 = IsotypicComponent(W1_023, 3)
print "V1_023(X,Y)"
show(character(W1_023))

generators = {D((5,0)) : [v023, v014], D((4,0)) : [v013], D((3,0)) : [v012]}
W1_tot = Subspace(generators=generators, operators=list_op, add_degrees=add_degree)

list_degrees = [tuple((i,j)) for i in range(0,deg+1) for j in range(0,deg+1) if i+j!=0 and i+j<deg+1]
sym_diff = symmetric_derivatives(r, list_degrees)
W2 = Range(W1, sym_diff, add_degrees=add_degrees_test)
W2_tot = Range(W1_tot, sym_diff, add_degrees=add_degrees_test)

W1 = IsotypicComponent(W1, 3)
print "V1(X,Y)"
show(character(W1))

W1_tot = IsotypicComponent(W1_tot, 3)
W2_tot = IsotypicComponent(W2_tot, 3)
print "V1_tot(X,Y)"
show(character(W1_tot))
print "diff"
show(character(W1) - character(W1_tot))

V1_014(X,Y)


s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + s[2] # s[3] + s[2, 1] # s[1, 1, 1] + s[2, 1] # s[2, 1] + s[3] # s[1, 1, 1] + 2*s[3] # s[2, 1] + s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[4] # s[2, 1] + s[5] # s[1, 1, 1]

V1_013(X,Y)


s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

V1_012(X,Y)


...

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + 2*s[2] # s[3] + s[2, 1] # s[1, 1, 1] + 2*s[2, 1] # s[2, 1] + s[3] # s[1, 1, 1] + 3*s[3] # s[2, 1] + s[3] # s[3] + 2*s[3, 1] # s[1, 1, 1] + s[3, 1] # s[2, 1] + s[4] # s[1, 1, 1] + 3*s[4] # s[2, 1] + s[4, 1] # s[1, 1, 1] + 2*s[5] # s[1, 1, 1] + s[5] # s[2, 1] + s[6] # s[1, 1, 1]

V1_tot(X,Y)


s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + s[1, 1] # s[1, 1, 1] + s[1, 1] # s[2, 1] + 2*s[2] # s[2, 1] + 2*s[2] # s[3] + s[2, 1] # s[1, 1, 1] + 2*s[2, 1] # s[2, 1] + s[3] # s[1, 1, 1] + 3*s[3] # s[2, 1] + 2*s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + 2*s[4] # s[2, 1] + 2*s[5] # s[1, 1, 1]

diff


s[3] # s[3] + s[3, 1] # s[2, 1] + s[4] # s[2, 1] + s[4, 1] # s[1, 1, 1] + s[5] # s[2, 1] + s[6] # s[1, 1, 1]

### Rencontre du 1er avril : $s_{22} \otimes s_{111}$

In [251]:
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]))
deg = v.degree()
generator = {v.multidegree() : [v]}
W1 = Subspace(generators=generator, operators={}, add_degrees=add_degree)
W1 = IsotypicComponent(W1, Partition([1,1,1]))

print "V1(X)"
show(character(W1))

r=2
op_pol = polarization_operators(r, max_deg=4)
#op_pol = merge(op_pol, higher_polarization_operators(P2))
W1 = PolarizedSpace(W1, op_pol)

print "V1(X,Y)"
show(character(W1))

V1(X)


s[6] # s[1, 1, 1]

V1(X,Y)


s[1, 1] # s[1, 1, 1] + s[2, 1] # s[1, 1, 1] + s[3] # s[1, 1, 1] + 2*s[3, 1] # s[1, 1, 1] + s[4] # s[1, 1, 1] + s[4, 1] # s[1, 1, 1] + s[5] # s[1, 1, 1] + s[6] # s[1, 1, 1]

**Vérification de l'indépendance linéaire des opérateurs.** But trouver celui qui va nous permettre d'obtenir $s_{22}$

In [252]:
v = vandermonde(Diagram([(0,0),(2,0),(4,0)]), r=2)
P1 = v.parent()
x = P1.derivative_variables()[0]
y = P1.derivative_variables()[1]

#grand vert = B
f1 = sum(y[i]*v.derivative(x[i]) for i in range(0,3))
B = sum(y[i]*f1.derivative(x[i], 3) for i in range(0,3))

#petit vert
f3 = sum(y[i]*v.derivative(x[i], 2) for i in range(0,3))
Z = sum(y[i]*f3.derivative(x[i], 2) for i in range(0,3))

#rouge = A
f5 = sum(y[i]*v.derivative(x[i]) for i in range(0,3))
f6 = sum(f5.derivative(x[i]) for i in range(0,3))
A = sum(y[i]*f6.derivative(x[i], 2) for i in range(0,3))

#bleu = C
f8 = sum(v.derivative(x[i]) for i in range(0,3))
f9 = sum(f8.derivative(x[i]) for i in range(0,3))
f10 = sum(y[i]*f9.derivative(x[i]) for i in range(0,3))
C = sum(y[i]*f10.derivative(x[i]) for i in range(0,3))

g1 = sum(x[i]*v.derivative(x[i], 2) for i in range(0,3))
g2 = sum(y[i]*g1.derivative(x[i], 2) for i in range(0,3))
D = sum(y[i]*g2.derivative(x[i]) for i in range(0,3))

#test element
g3 = sum(y[i]*v.derivative(x[i], 2) for i in range(0,3))
E = sum(x[i]*y[i]*g3.derivative(x[i], 3) for i in range(0,3))

show(E)

S = Subspace([A, B, C, D, E])
show(S.basis())
S.dimension()

-48*x00^2*x10*x11 + 48*x01^2*x10*x11 + 48*x00^2*x10*x12 - 48*x02^2*x10*x12 - 48*x01^2*x11*x12 + 48*x02^2*x11*x12

{0: (-x01^2*x10^2 + x02^2*x10^2 + x00^2*x11^2 - x02^2*x11^2 - x00^2*x12^2 + x01^2*x12^2,
  -x00*x01*x10^2 + x00*x02*x10^2 + x00*x01*x11^2 - x01*x02*x11^2 - x00*x02*x12^2 + x01*x02*x12^2,
  x00^2*x10*x11 - x01^2*x10*x11 - x00^2*x10*x12 + x02^2*x10*x12 + x01^2*x11*x12 - x02^2*x11*x12)}

3

### Cas du partage 311

In [253]:
v = vandermonde(Partition([3,1,1]))
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(2, max_deg=deg_v)
W2 = PolarizedSpace(IsotypicComponent(W1, 5), op_pol)
character(W2)

s[] # s[3, 1, 1] + s[1] # s[2, 1, 1, 1] + s[1] # s[2, 2, 1] + s[1, 1] # s[1, 1, 1, 1, 1] + s[2] # s[2, 1, 1, 1] + s[3] # s[1, 1, 1, 1, 1]

### Exemples avec et sans les antisymétries

Comparons les bases sur un petit exemple sans puis avec l'utilisation des antisymétries.

In [254]:
for nu in Partitions(3):
    print nu
    show(vandermonde(nu))
    show(antisymmetries(nu))

[3]


-x00^2*x01 + x00*x01^2 + x00^2*x02 - x01^2*x02 - x00*x02^2 + x01*x02^2

((0,), (1,), (2,))

[2, 1]


-x01*theta00 + x02*theta00 + x00*theta01 - x02*theta01 - x00*theta02 + x01*theta02

((0, 2), (1,))

[1, 1, 1]


-theta00^2*theta01 + theta00*theta01^2 + theta00^2*theta02 - theta01^2*theta02 - theta00*theta02^2 + theta01*theta02^2

((0, 1, 2),)

In [255]:
mu = Partition([2,1])
n = 3
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(2, deg_v)
W2 = PolarizedSpace(IsotypicComponent(W1, n), op_pol)
show(character(W2))
for v in W2.values():
    show(v.basis())

s[] # s[2, 1] + s[1] # s[1, 1, 1]

{((0, 1),
  (1,
   1,
   1)): (-x11*theta00 + x12*theta00 + x10*theta01 - x12*theta01 - x10*theta02 + x11*theta02,),
 ((1, 0),
  (1,
   1,
   1)): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

{((0, 0), (2, 1)): (theta00 - theta02,)}

In [256]:
mu = Partition([2,1])
n = 3
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, deg_v)
W2 = PolarizedSpace(IsotypicComponent(W1, n, use_antisymmetry=True), op_pol)
show(character(W2))
for v in W2.values():
    show(v.basis())

s[] # s[2, 1] + s[1] # s[1, 1, 1]

{((0, 1), (1, 1, 1)): (x10*theta01,), ((1, 0), (1, 1, 1)): (x00*theta01,)}

{((0, 0), (2, 1)): (theta00,)}

Comparaison des temps de calcul avec et sans les antisymétries sur un exemple plus gros

In [257]:
import datetime

In [258]:
t1 = datetime.datetime.now()
mu = Partition([3,1])
n = mu.size()
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, deg_v)
W2 = PolarizedSpace(IsotypicComponent(W1, n), op_pol)
show(character(W2))
t2 = datetime.datetime.now()
show(t2-t1)

s[] # s[3, 1] + s[1] # s[2, 1, 1] + s[1] # s[2, 2] + s[1, 1] # s[1, 1, 1, 1] + s[2] # s[2, 1, 1] + s[3] # s[1, 1, 1, 1]

datetime.timedelta(0, 1, 667741)

In [259]:
t1 = datetime.datetime.now()
mu = Partition([3,1])
n = mu.size()
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, deg_v)
W2 = PolarizedSpace(IsotypicComponent(W1, n, use_antisymmetry=True), op_pol)
show(character(W2))
t2 = datetime.datetime.now()
show(t2-t1)

s[] # s[3, 1] + s[1] # s[2, 1, 1] + s[1] # s[2, 2] + s[1, 1] # s[1, 1, 1, 1] + s[2] # s[2, 1, 1] + s[3] # s[1, 1, 1, 1]

datetime.timedelta(0, 0, 735903)

### Exemples avec et sans les symétries sur les lignes

Comparons les bases sur un petit exemple sans puis avec l'utilisation des symétries sur les lignes.

In [260]:
mu = Partition([2,1,1])
n = mu.size()
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, deg_v)
W2 = PolarizedSpace(IsotypicComponent(W1, n), op_pol)
show(character(W2))
for v in W2.values():
    show(v.basis())

s[] # s[2, 1, 1] + s[1] # s[1, 1, 1, 1]

{((0, 0, 0),
  (2,
   1,
   1)): (theta00^2*theta02 - theta00*theta02^2 - theta00^2*theta03 + theta02^2*theta03 + theta00*theta03^2 - theta02*theta03^2,)}

{((0, 0, 1),
  (1,
   1,
   1,
   1)): (-x22*theta00^2*theta01 + x23*theta00^2*theta01 + x22*theta00*theta01^2 - x23*theta00*theta01^2 + x21*theta00^2*theta02 - x23*theta00^2*theta02 - x20*theta01^2*theta02 + x23*theta01^2*theta02 - x21*theta00*theta02^2 + x23*theta00*theta02^2 + x20*theta01*theta02^2 - x23*theta01*theta02^2 - x21*theta00^2*theta03 + x22*theta00^2*theta03 + x20*theta01^2*theta03 - x22*theta01^2*theta03 - x20*theta02^2*theta03 + x21*theta02^2*theta03 + x21*theta00*theta03^2 - x22*theta00*theta03^2 - x20*theta01*theta03^2 + x22*theta01*theta03^2 + x20*theta02*theta03^2 - x21*theta02*theta03^2,),
 ((0, 1, 0),
  (1,
   1,
   1,
   1)): (-x12*theta00^2*theta01 + x13*theta00^2*theta01 + x12*theta00*theta01^2 - x13*theta00*theta01^2 + x11*theta00^2*theta02 - x13*theta00^2*theta02 - x10*theta01^2*theta02 + x13*theta01^2*theta02 - x11*theta00*theta02^2 + x13*theta00*theta02^2 + x10*theta01*theta02^2 - x13*theta01*theta02^2 - x11*theta00^2*theta03 + x12*theta00^2*theta03 + x10*theta01^2*theta03 - x12*theta01^2*theta03 - x10*theta02^2*theta03 + x11*theta02^2*theta03 + x11*theta00*theta03^2 - x12*theta00*theta03^2 - x10*theta01*theta03^2 + x12*theta01*theta03^2 + x10*theta02*theta03^2 - x11*theta02*theta03^2,),
 ((1, 0, 0),
  (1,
   1,
   1,
   1)): (-x02*theta00^2*theta01 + x03*theta00^2*theta01 + x02*theta00*theta01^2 - x03*theta00*theta01^2 + x01*theta00^2*theta02 - x03*theta00^2*theta02 - x00*theta01^2*theta02 + x03*theta01^2*theta02 - x01*theta00*theta02^2 + x03*theta00*theta02^2 + x00*theta01*theta02^2 - x03*theta01*theta02^2 - x01*theta00^2*theta03 + x02*theta00^2*theta03 + x00*theta01^2*theta03 - x02*theta01^2*theta03 - x00*theta02^2*theta03 + x01*theta02^2*theta03 + x01*theta00*theta03^2 - x02*theta00*theta03^2 - x00*theta01*theta03^2 + x02*theta01*theta03^2 + x00*theta02*theta03^2 - x01*theta02*theta03^2,)}

In [261]:
mu = Partition([2,1,1])
n = 4
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, max_deg=deg_v, row_symmetry="permutation")
W2 = PolarizedSpace(IsotypicComponent(W1, n), op_pol, add_degrees=add_degrees_symmetric)
show(character(W2, row_symmetry="permutation"))
for v in W2.values():
    show(v.basis())

s[] # s[2, 1, 1] + s[1] # s[1, 1, 1, 1]

{((0, 0, 0),
  (2,
   1,
   1)): (theta00^2*theta02 - theta00*theta02^2 - theta00^2*theta03 + theta02^2*theta03 + theta00*theta03^2 - theta02*theta03^2,)}

{((1, 0, 0),
  (1,
   1,
   1,
   1)): (-x02*theta00^2*theta01 + x03*theta00^2*theta01 + x02*theta00*theta01^2 - x03*theta00*theta01^2 + x01*theta00^2*theta02 - x03*theta00^2*theta02 - x00*theta01^2*theta02 + x03*theta01^2*theta02 - x01*theta00*theta02^2 + x03*theta00*theta02^2 + x00*theta01*theta02^2 - x03*theta01*theta02^2 - x01*theta00^2*theta03 + x02*theta00^2*theta03 + x00*theta01^2*theta03 - x02*theta01^2*theta03 - x00*theta02^2*theta03 + x01*theta02^2*theta03 + x01*theta00*theta03^2 - x02*theta00*theta03^2 - x00*theta01*theta03^2 + x02*theta01*theta03^2 + x00*theta02*theta03^2 - x01*theta02*theta03^2,)}

Avec les antisymétries aussi. 

In [262]:
mu = Partition([2,1,1])
n = 4
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, max_deg=deg_v, row_symmetry="permutation")
W2 = PolarizedSpace(IsotypicComponent(W1, n, use_antisymmetry=True), op_pol, add_degrees=add_degrees_symmetric)
show(character(W2, row_symmetry="permutation"))
for v in W2.values():
    show(v.basis())

s[] # s[2, 1, 1] + s[1] # s[1, 1, 1, 1]

{((0, 0, 0), (2, 1, 1)): (theta00^2*theta02,)}

{((1, 0, 0), (1, 1, 1, 1)): (x00*theta01^2*theta02,)}

Comparaison des temps de calcul avec et sans les symétries sur un exemple plus gros

In [263]:
import datetime

In [264]:
t1 = datetime.datetime.now()
mu = Partition([3,1])
n = mu.size()
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, deg_v)
W2 = PolarizedSpace(IsotypicComponent(W1, n), op_pol)
show(character(W2))
t2 = datetime.datetime.now()
show(t2-t1)

s[] # s[3, 1] + s[1] # s[2, 1, 1] + s[1] # s[2, 2] + s[1, 1] # s[1, 1, 1, 1] + s[2] # s[2, 1, 1] + s[3] # s[1, 1, 1, 1]

datetime.timedelta(0, 1, 772244)

In [265]:
t1 = datetime.datetime.now()
mu = Partition([3,1])
n = mu.size()
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, max_deg=deg_v, row_symmetry="permutation")
W2 = PolarizedSpace(IsotypicComponent(W1, n), op_pol, add_degrees=add_degrees_symmetric)
show(character(W2, row_symmetry="permutation"))
t2 = datetime.datetime.now()
show(t2-t1)

s[] # s[3, 1] + s[1] # s[2, 1, 1] + s[1] # s[2, 2] + s[1, 1] # s[1, 1, 1, 1] + s[2] # s[2, 1, 1] + s[3] # s[1, 1, 1, 1]

datetime.timedelta(0, 1, 209982)

In [266]:
t1 = datetime.datetime.now()
mu = Partition([3,1])
n = mu.size()
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, max_deg=deg_v, row_symmetry="permutation")
W2 = PolarizedSpace(IsotypicComponent(W1, n, use_antisymmetry=True), op_pol, add_degrees=add_degrees_symmetric)
show(character(W2, row_symmetry="permutation"))
t2 = datetime.datetime.now()
show(t2-t1)

s[] # s[3, 1] + s[1] # s[2, 1, 1] + s[1] # s[2, 2] + s[1, 1] # s[1, 1, 1, 1] + s[2] # s[2, 1, 1] + s[3] # s[1, 1, 1, 1]

datetime.timedelta(0, 0, 584504)

### Cas $m=4$, $n=6$

In [267]:
v = vandermonde(Diagram([(0,0),(1,0),(2,0),(3,0),(0,1),(2,1)]))
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(3, max_deg=deg_v, row_symmetry="permutation")
#W2 = PolarizedSpace(IsotypicComponent(W1, 6, use_antisymmetry=True), op_pol)
#character(W2, row_symmetry="permutation")

### Autres cas

#### Diagramme $\{(0,0),(0,1),(1,1),(0,2)\}$ et autres

Pas de quotient dans ce cas.

In [268]:
n = 4
gamma = Partition([2,2])
v = vandermonde(gamma)
show(v)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
show(character(IsotypicComponent(W1, n, use_antisymmetry=True)))

r = 3
op_pol = polarization_operators(r, deg_v, row_symmetry="permutation")
W1 = PolarizedSpace(IsotypicComponent(W1, n), op_pol)
character(W1, row_symmetry="permutation")

x00*x02*theta00*theta01 - x01*x02*theta00*theta01 - x00*x03*theta00*theta01 + x01*x03*theta00*theta01 - x00*x01*theta00*theta02 + x01*x02*theta00*theta02 + x00*x03*theta00*theta02 - x02*x03*theta00*theta02 + x00*x01*theta01*theta02 - x00*x02*theta01*theta02 - x01*x03*theta01*theta02 + x02*x03*theta01*theta02 + x00*x01*theta00*theta03 - x00*x02*theta00*theta03 - x01*x03*theta00*theta03 + x02*x03*theta00*theta03 - x00*x01*theta01*theta03 + x01*x02*theta01*theta03 + x00*x03*theta01*theta03 - x02*x03*theta01*theta03 + x00*x02*theta02*theta03 - x01*x02*theta02*theta03 - x00*x03*theta02*theta03 + x01*x03*theta02*theta03

s[] # s[2, 2] + s[1] # s[2, 1, 1] + s[2] # s[1, 1, 1, 1]

s[] # s[2, 2] + s[1] # s[2, 1, 1] + s[2] # s[1, 1, 1, 1]

In [269]:
n = 5
gamma = Partition([2,2,1])
v = vandermonde(gamma)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
show(character(IsotypicComponent(W1, n, use_antisymmetry=True)))

r = 4
op_pol = polarization_operators(r, deg_v, row_symmetry="permutation")
W1 = PolarizedSpace(IsotypicComponent(W1, n, use_antisymmetry=True), op_pol)
character(W1, row_symmetry="permutation")

s[] # s[2, 2, 1] + s[1] # s[2, 1, 1, 1] + s[2] # s[1, 1, 1, 1, 1]

s[] # s[2, 2, 1] + s[1] # s[2, 1, 1, 1] + s[2] # s[1, 1, 1, 1, 1]

In [270]:
gamma = Partition([2,2])
n = gamma.size()
v = vandermonde(gamma)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
character(IsotypicComponent(W1, n, use_antisymmetry=True))

op_pol = polarization_operators(3, deg_v, row_symmetry="permutation")
W1 = PolarizedSpace(IsotypicComponent(W1, n, use_antisymmetry=True), op_pol)
factorise(character(W1, row_symmetry="permutation"), 4)

[1, 1, 1, 1]


s[2]

[2, 2]


s[]

[2, 1, 1]


s[1]

### Implémentation formule de François

Ne fonctionne que pour un jeu de variables.

In [271]:
Sym = SymmetricFunctions(FractionField(QQ['q','t']))
Sym.inject_shorthands(verbose=False)
Ht = Sym.macdonald().Ht(); Ht

Symmetric Functions over Fraction Field of Multivariate Polynomial Ring in q, t over Rational Field in the Macdonald Ht basis

In [272]:
s(Ht([1,1,1]))

t^3*s[1, 1, 1] + (t^2+t)*s[2, 1] + s[3]

In [273]:
s(Ht([2,1]))

q*t*s[1, 1, 1] + (q+t)*s[2, 1] + s[3]

In [274]:
s(Ht([3]))

q^3*s[1, 1, 1] + (q^2+q)*s[2, 1] + s[3]